In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import tensorflow as tf
import sys
import time

# --- 설정 변수 정의 ---
# 작업 디렉토리 정의
# 학습 데이터 파일명
datafile = "/src/data/DataSet/FFTData.npy"
# 진동 분석 모델 디렉토리명
modeldir = "/src/hyebin/model/FFT-01"
# tensorboard용 log 디렉토리명
tblogdir = "/src/hyebin/logs"

# 학습 데이터 설정
# 학습 데이터 중 최초 데이터 위치
base_index = 2000
# 학습할 데이터의 개수
training_count = 1000

# 전결합층 설정
# 전결합층 첫번째 레이어 unit 개수
layer1_unit_count = 1024
# 전결합층 두번째 레이어 unit 개수
layer2_unit_count = 1024
# 전결합층 두번째 레이어 unit 개수
layer3_unit_count = 1024
# 전결합층 dropout 비율
dropout_rate = 0.4

# 최적화 함수 설정
# 최적화 함수 학습률
learning_rate = 0.0001

# 학습 수행 설정
# 최대 학습 수행 회수
epoch_count = 5000
# 학습 중단 시킬 최소 loss값
loss_limit = 0.0001


# --- 학습 데이터 로딩  ---
# 데이터 구조
# data[0] : 학습용 데이터(이미지 개수, 36, 48, 3)
# data[1] : 테스트용 데이터(이미지 개수, 36, 48, 3)
# data[2] : 학습용 데이터의 라벨(이미지 개수, 5)
# data[3] : 테스트용 데이터의 라벨 (이미지 개수, 5)
print("load data : ", datafile)
data = np.load(datafile, allow_pickle=True)
print("load complete")

# 학습용 데이터와 라벨을 사용함
train_x = data[0][base_index:base_index + training_count]
train_y = data[2][base_index:base_index + training_count]

# --- VGG Net 신경망 구성 ---
# 입력층 : 36*48 의 Spectrogram 전처리 데이터
# 특징 추출층 : 2개 합성곱 필터, 1개의 필터를 2회 수행 
# 전 결합층 : 3개의 은닉층(층당 유닛 1024개) -> 단층에서 발생하는 xor 문제 회피, 1개의 출력층(유닛 5개)
use_dropout = tf.placeholder(tf.bool, name="use_dropout")

# --- 입력층 ---
x = tf.placeholder(tf.float32, [None, 36, 48, 3], name="in")

# --- 특징 추출층 ---
conv11 = tf.layers.conv2d(
    inputs=x,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv12 = tf.layers.conv2d(
    inputs=conv11,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# FFT : 36 * 48 → 18 * 24
pool1 = tf.layers.max_pooling2d(
    inputs=conv12,
    pool_size=[2, 2],
    strides=2)

conv21 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

conv22 = tf.layers.conv2d(
    inputs=conv21,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

# 풀링으로 사이즈가 줄어듦
# FFT : 18 * 24 → 9 * 12
pool2 = tf.layers.max_pooling2d(
    inputs=conv22,
    pool_size=[2, 2],
    strides=2)

# --- 전 결합층 ---
# 데이터 행렬 평탄화(flatten)
# FFT : 9 * 12(이미지사이즈) * 64(합성곱 필터 수)
input_cnt = 9 * 12 * 64
pool_flat = tf.reshape(pool2, [-1, input_cnt])

dense1 = tf.layers.dense(inputs=pool_flat, units=layer1_unit_count, activation=tf.nn.relu)
dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate, training=use_dropout)

dense2 = tf.layers.dense(inputs=dropout1, units=layer2_unit_count, activation=tf.nn.relu)
dropout2 = tf.layers.dropout(inputs=dense2, rate=dropout_rate, training=use_dropout)

dense3 = tf.layers.dense(inputs=dropout2, units=layer3_unit_count, activation=tf.nn.relu)
dropout3 = tf.layers.dropout(inputs=dense3, rate=dropout_rate, training=use_dropout)

netout = tf.layers.dense(inputs=dropout3, units=5)
softout = tf.nn.softmax(netout, name="out")

# --- loss 함수 정의 ---
# 평균 제곱 오차 함수 사용
y = tf.placeholder(tf.float32, [None, 5])
loss = tf.reduce_mean(tf.reduce_sum(tf.square(softout - y), 1))
loss_summ = tf.summary.scalar("loss", loss)

# --- loss 함수의 최적화 함수 정의 ---
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# --- 학습 수행 ---
with tf.Session() as sess:
    # tensorboard용 로깅 정의    
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(tblogdir)
    writer.add_graph(sess.graph)
    
    # 신경망 가중치(weight) 초기화
    sess.run(tf.global_variables_initializer())
    
    # 전체 데이터에 대해 5000번 수행
    # loss가 0.001 이하일 때, 학습 중단
    start_time = time.time()
    for epoch in range(epoch_count):
        _train, _loss_summ, _loss = sess.run([train, loss_summ, loss], feed_dict={x:train_x, y:train_y, use_dropout:True})
        writer.add_summary(summary = _loss_summ, global_step = epoch)
        
        if epoch % 100 == 0:
            print("index=", epoch, "  loss=", _loss)
            
        if _loss < loss_limit:
            print("index=", epoch, "  loss=", _loss)
            break
            
    end_time = time.time()
    print("running time : ", end_time - start_time)
    
    # --- 학습 모델 저장 ----
    print("save model")
    builder = tf.saved_model.builder.SavedModelBuilder(modeldir)
    signature = tf.saved_model.predict_signature_def(inputs={"in":x}, outputs={"out":softout})
    builder.add_meta_graph_and_variables(sess, tags=["ver1"], signature_def_map={"escalator-vibration-analysis-v1":signature})
    builder.save()
    print("complete")
    

load data :  /src/data/DataSet/FFTData.npy
load complete
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
index= 0   loss= 0.8019243
index= 100   loss= 0.4538477
index= 200   loss= 0.12017423
index= 300   loss= 0.037116613
index= 400   loss= 0.015008687
index= 500   loss= 0.009524986
index= 600   loss= 0.005668774
index= 700   loss= 0.008854443
index= 800   loss= 0.0005409646
index= 900   loss= 0.0004157069
index= 1000   loss= 0.00030895442
index= 1100   loss= 0.00052108953
index= 1108   loss= 8.528254e-05
running time :  93.74000525474548
save model
Instructio